In [ ]:
# Importing PlaidML
import plaidml.keras
plaidml.keras.install_backend()
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

In [ ]:
import os
import numpy as np
import glob
import shutil
import matplotlib.pyplot as plt

In [ ]:
#import packages
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#load data

_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

zip_file = tf.keras.utils.get_file(origin=_URL,
                                   fname="flower_photos.tgz",
                                   extract=True)

base_dir = os.path.join(os.path.dirname(zip_file), 'flower_photos')

In [ ]:
#label classes
classes = ['roses', 'daisy', 'dandelion', 'sunflowers', 'tulips']

In [ ]:
#print total number of each type of flower
for cl in classes:
  img_path = os.path.join(base_dir, cl)
  images = glob.glob(img_path + '/*.jpg')
  print("{}: {} Images".format(cl, len(images)))
  train, val = images[:round(len(images)*0.8)], images[round(len(images)*0.8):]

  for t in train:
    if not os.path.exists(os.path.join(base_dir, 'train', cl)):
      os.makedirs(os.path.join(base_dir, 'train', cl))
    shutil.move(t, os.path.join(base_dir, 'train', cl))

  for v in val:
    if not os.path.exists(os.path.join(base_dir, 'val', cl)):
      os.makedirs(os.path.join(base_dir, 'val', cl))
    shutil.move(v, os.path.join(base_dir, 'val', cl))

In [ ]:
#set training and validation sets
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

In [ ]:
#training dataset by flower
train_roses_dir = os.path.join(train_dir, 'roses')
train_daisy_dir = os.path.join(train_dir, 'daisy')
train_dand_dir = os.path.join(train_dir, 'dandelion')
train_sunfl_dir= os.path.join(train_dir, 'sunflowers')
train_tulips_dir = os.path.join(train_dir, 'tulips')

num_roses_tr = len(os.listdir(train_roses_dir))
num_daisy_tr = len(os.listdir(train_daisy_dir))
num_dand_tr = len(os.listdir(train_dand_dir))
num_sunfl_tr = len(os.listdir(train_sunfl_dir))
num_tulips_tr = len(os.listdir(train_tulips_dir))

print('total training rose images', num_roses_tr)
print('total training daisy images', num_daisy_tr)
print('total training dand images', num_dand_tr)
print('total training sunfl images', num_sunfl_tr)
print('total training tulips images', num_tulips_tr)

In [ ]:
#validation dataset by flower
val_roses_dir = os.path.join(val_dir, 'roses')
val_daisy_dir = os.path.join(val_dir, 'daisy')
val_dand_dir = os.path.join(val_dir, 'dandelion')
val_sunfl_dir= os.path.join(val_dir, 'sunflowers')
val_tulips_dir = os.path.join(val_dir, 'tulips')

num_roses_val = len(os.listdir(val_roses_dir))
num_daisy_val = len(os.listdir(val_daisy_dir))
num_dand_val = len(os.listdir(val_dand_dir))
num_sunfl_val = len(os.listdir(val_sunfl_dir))
num_tulips_val = len(os.listdir(val_tulips_dir))

print('total val rose images', num_roses_val)
print('total val daisy images', num_daisy_val)
print('total val dand images', num_dand_val)
print('total val sunfl images', num_sunfl_val)
print('total val tulips images', num_tulips_val)

In [ ]:
#set total datasets as variables
total_train = num_roses_tr + num_daisy_tr + num_dand_tr + num_sunfl_tr + num_tulips_tr 
print("Total training imgages", total_train)

total_val = num_roses_val + num_daisy_val + num_dand_val + num_sunfl_val + num_tulips_val 
print("Total val imgages", total_val)

In [ ]:
#data augmentation

batch_size = 100
IMG_SHAPE = 150

In [ ]:
#horizontally flip images to augment
image_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_SHAPE, IMG_SHAPE))

In [ ]:
#view flipped image example
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()


augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

In [ ]:
#apply random image rotation to augment
image_gen = ImageDataGenerator(rescale=1./255, rotation_range=45)

train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_SHAPE, IMG_SHAPE))

In [ ]:
#view rotated image example
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

In [ ]:
#apply image zoom to augment
image_gen = ImageDataGenerator(rescale=1./255, zoom_range=0.5)

train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_SHAPE,IMG_SHAPE))

In [ ]:
#view zoomed image example
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

In [ ]:
#use all 3 augmentation strategies
image_gen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    zoom_range=0.5,
    horizontal_flip=True,
    width_shift_range=0.15,
    height_shift_range=0.15,
)


train_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,
                                                     directory=train_dir,
                                                     shuffle=True,
                                                     target_size=(IMG_SHAPE,IMG_SHAPE),
                                                     class_mode='sparse')

In [ ]:
#view example of all augmentation strategies
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

In [ ]:
#create image generator for validation data
image_gen_val = ImageDataGenerator(rescale=1./255)
val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size,
                                                 directory=val_dir,
                                                 target_size=(IMG_SHAPE,IMG_SHAPE),
                                                 class_mode='sparse')

In [ ]:
#build CNN
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), padding='same', activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512, activation='relu'),

    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(5)
])

In [ ]:
#compile the model
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
#train the model
epochs = 80

history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(total_train/float(batch_size))),
    epochs = epochs,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(total_val/float(batch_size)))
)

In [ ]:
#plot training and validation outcomes
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()